In [2]:
import random
import spacy
from nltk.tokenize import wordpunct_tokenize

nlp_en = spacy.load('en')
nlp_sp = spacy.load('es')

In [18]:
def read_tables(fname, falign):
    # with open("data/chat_prev/en_rule.txt") as f:
    #     en_orig = [line.replace('\n','') for line in f.readlines()]
    # with open("data/chat_prev/sp_rule.txt") as f:
    #     sp_orig = [line.replace('\n','') for line in f.readlines()]

    en_orig = []
    sp_orig = []
    with open(fname) as f:
        for line in f.readlines():
            items = line.replace('\n','').split(' ||| ')
            en_orig.append(items[0])
            sp_orig.append(items[1])
            
    en_idx = []
    sp_idx = []
    with open(falign) as f:
        for line in f.readlines():
            items = line.replace('\n','').split()
            en_idx.append([int(pair.split('-')[0]) for pair in items])
            sp_idx.append([int(pair.split('-')[1]) for pair in items])
        
    return dict([('en', en_orig), ('sp', sp_orig), ('en_idx', en_idx), ('sp_idx', sp_idx)])

tables_dct = read_tables("en-sp_rule.txt", "aligned.gdfa")
print tables_dct['sp'][3]

¿ tienes alguna etiqueta láser o español ? 


In [3]:
sent_sp = "me gusta nadar. yo soy una estudiante"
sent_en = "I like swimming and i study islamic studies"
doc_sp = nlp_sp(unicode(sent_sp, "utf-8"))
doc_en = nlp_en(unicode(sent_en, "utf-8"))

# chunks = list(doc_sp.noun_chunks)
# print [chunk.text for chunk in chunks]

# print [(en_token.text, en_token.pos_) for en_token in doc_en]
# print [(en_token.text, en_token.pos_) for en_token in doc_sp]

In [22]:
for i in range(5):
    en_txt = tables_dct['en'][i]
    sp_txt = tables_dct['sp'][i]
    en_words = en_txt.split()
    sp_words = sp_txt.split()
    en_idxs = tables_dct['en_idx'][i]
    sp_idxs = tables_dct['sp_idx'][i]
    
    en_doc = nlp_en(unicode(en_txt, "utf-8"))
    sp_doc = nlp_sp(unicode(sp_txt, "utf-8"))
    
    for noun_chunk in en_doc.noun_chunks: # noun_chunk type = Span
        chunk_start = noun_chunk.start
        chunk_end = noun_chunk.end
        
        
    
    for j, e_idx in enumerate(tables_dct['en_idx'][i]):
        s_idx = s_idxs[j]
#         print en_words[e_idx], sp_words[s_idx]
#     print "*"*10
    

NameError: name 'en_txt' is not defined

In [ ]:
# from spacy import displacy
# displacy.serve(doc_sp, style='dep')

In [ ]:
def sp_matrix_eng_nouns(en_txt, idx): #tables_dct, 
    """ en_txt: string of words (not list)
        tables_dct: contains 2 tables, keys = en,sp
        idx: int of which index in table is this sentence
        [return] string of words (list separated by space)
    """
    en_txt = tables_dct['en'][idx]
    en_doc = nlp_en(unicode(en_txt, "utf-8"))
#     en_nouns = [en_token.text for en_token in en_doc if en_token.pos_ in ['NOUN', 'PROPN']]
    # avoid "who"
#     en_nouns = [noun for noun in en_nouns if not noun==u"who"]
    # print "EN NOUNS", en_nouns

    sp_txt = tables_dct['sp'][idx]
    sp_doc = nlp_sp(unicode(sp_txt, "utf-8"))
    sp_nouns = []
    for orig_i, sp_token in enumerate(sp_doc):
        if sp_token.pos_ in ['NOUN', 'PROPN']: sp_nouns.append((sp_token.text, orig_i))
        # capture infinitive verbs ==> turn into noun (e.g. nadar = swimming)
        if sp_token.pos_=='VERB' and sp_token.tag_.endswith('Inf'): sp_nouns.append((sp_token.text, orig_i))
        
    # print "SP NOUNS", sp_nouns

    cm_txt = [sp_token.text for sp_token in sp_doc]
    for noun_idx, (sp_token, orig_i) in enumerate(sp_nouns):
        if not en_nouns: break
        if noun_idx < len(en_nouns)-1: #valid, or else is last
            cm_txt[orig_i] = en_nouns[noun_idx]
        else: # get last noun (probably last one anyway??)
            cm_txt[orig_i] = en_nouns[-1]
    return " ".join(cm_txt)

In [ ]:
def old_sp_matrix_eng_nouns(en_txt, idx): #tables_dct, 
    """ en_txt: string of words (not list)
        tables_dct: contains 2 tables, keys = en,sp
        idx: int of which index in table is this sentence
        [return] string of words (list separated by space)
    """
    en_doc = nlp_en(unicode(en_txt, "utf-8"))
    en_nouns = [en_token.text for en_token in en_doc if en_token.pos_ in ['NOUN', 'PROPN']]
    # avoid "who"
    en_nouns = [noun for noun in en_nouns if not noun==u"who"]
    # print "EN NOUNS", en_nouns

    sp_txt = tables_dct['sp'][idx]
    sp_doc = nlp_sp(unicode(sp_txt, "utf-8"))
    sp_nouns = []
    for orig_i, sp_token in enumerate(sp_doc):
        if sp_token.pos_ in ['NOUN', 'PROPN']: sp_nouns.append((sp_token.text, orig_i))
        # capture infinitive verbs ==> turn into noun (e.g. nadar = swimming)
        if sp_token.pos_=='VERB' and sp_token.tag_.endswith('Inf'): sp_nouns.append((sp_token.text, orig_i))
        
    # print "SP NOUNS", sp_nouns

    cm_txt = [sp_token.text for sp_token in sp_doc]
    for noun_idx, (sp_token, orig_i) in enumerate(sp_nouns):
        if not en_nouns: break
        if noun_idx < len(en_nouns)-1: #valid, or else is last
            cm_txt[orig_i] = en_nouns[noun_idx]
        else: # get last noun (probably last one anyway??)
            cm_txt[orig_i] = en_nouns[-1]
    return " ".join(cm_txt)

In [ ]:
sent_sp = "Tengo dos amigos que estudiaron estudios islámicos."
sent_en = "I have two friends who studied islamic studies."

In [ ]:
def print_lex():
	print "STYLE 1: LEXICAL on Rule-based Bot // 6 Feb 2018"
	print "KEY:\n1. Spanish (orig)\n2. English (orig)\n3. CodeMix"
	print "*"*10

	for i, en_sent in enumerate(tables_dct['en']):
		print tables_dct['sp'][i]
		print en_sent
		print sp_matrix_eng_nouns(en_sent, i).encode('utf-8') # encoding req'd to save print logs
		print "*"*10
		# if i==10: break